In [ ]:
!nvidia-smi

Sat Jun 20 15:10:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# %%writefile setup.sh

!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 7274 (delta 9), reused 6 (delta 0), pack-reused 7255
Receiving objects: 100% (7274/7274), 13.87 MiB | 14.39 MiB/s, done.
Resolving deltas: 100% (4909/4909), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-1v8m3oe1
Created temporary directory: /tmp/pip-req-tracker-99efx_na
Created requirements tracker '/tmp/pip-req-tracker-99efx_na'
Created temporary directory: /tmp/pip-install-ohxn3okr
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-xl6bx3o1
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-99efx_na'
    Running setup.py (path:/t

In [ ]:
!pip install simpletransformers
import simpletransformers
from simpletransformers.classification import ClassificationModel
import pandas as pd
import sklearn
import logging
import time

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

     |████████████████████████████████| 194kB 5.9MB/s 
     |████████████████████████████████| 3.8MB 14.0MB/s 
     |████████████████████████████████| 204kB 35.8MB/s 
     |████████████████████████████████| 675kB 40.0MB/s 
     |████████████████████████████████| 890kB 35.5MB/s 
     |████████████████████████████████| 1.1MB 35.2MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=7d170c43fc9aad79c1b87f20137aab5d7e756d326423172a34df565bed1afc08
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=d2d49dbbe459f939252791cb340d7867073501e8b28fbe4ef84d65425dd88fad
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses


In [ ]:
train_df = pd.read_csv("train.tsv", delimiter="\t", names=["text_a", "text_b", "labels"], dtype="object")
train_df["labels"] = train_df["labels"].astype(int)

train_set = train_df

eval_set = pd.read_csv("dev.tsv", delimiter="\t", names=["text_a", "text_b", "labels"], dtype="object")
eval_set["labels"] = eval_set["labels"].astype(int)

In [ ]:
!mkdir spanbert
!wget https://cdn.huggingface.co/SpanBERT/spanbert-base-cased/pytorch_model.bin -P spanbert
!wget https://s3.amazonaws.com/models.huggingface.co/bert/SpanBERT/spanbert-base-cased/config.json -P spanbert
!wget https://cdn.huggingface.co/SpanBERT/spanbert-base-cased/vocab.txt -P spanbert

--2020-06-20 15:28:24--  https://cdn.huggingface.co/SpanBERT/spanbert-base-cased/pytorch_model.bin
Resolving cdn.huggingface.co (cdn.huggingface.co)... 143.204.198.23, 143.204.198.28, 143.204.198.84, ...
Connecting to cdn.huggingface.co (cdn.huggingface.co)|143.204.198.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215475882 (205M) [application/octet-stream]
Saving to: ‘spanbert/pytorch_model.bin’

pytorch_model.bin   100%[===================>] 205.49M  49.5MB/s    in 4.5s    

2020-06-20 15:28:29 (45.8 MB/s) - ‘spanbert/pytorch_model.bin’ saved [215475882/215475882]

--2020-06-20 15:28:30--  https://s3.amazonaws.com/models.huggingface.co/bert/SpanBERT/spanbert-base-cased/config.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.165.165
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.165.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 413 [application/json]
Saving to: ‘spanbert/config.json’

config.json     

In [ ]:
import torch
train_args={
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 3,
    # 'evaluate_during_training' : True,
    'train_batch_size': 32,
    'eval_batch_size': 32,
    'save_eval_checkpoints' : False,
    'save_model_every_epoch' : False,
    'use_cached_eval_features' : True,
    "save_steps": 0,
    'tensorboard_dir' : "/content/outputs/",
    # "evaluate_during_training_verbose": True,
    'silent' : True,
}
# # # Create a ClassificationModel
model = ClassificationModel('bert', '/content/spanbert', num_labels=2, use_cuda=True, cuda_device=0, args=train_args)


stime = time.time()
# # # # Train the model
model.train_model(train_set, eval_df=eval_set)
# model2 = ClassificationModel('bert', '/content/outputs', num_labels=2, use_cuda=True, cuda_device=0, args=train_args)
# # Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_set, acc=sklearn.metrics.accuracy_score)
print(time.time()-stime)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model:   Starting fine-tuning.


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:114: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss 

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model:{'mcc': 0.8038802905308874, 'tp': 13109, 'tn': 23568, 'fp': 1946, 'fn': 1748, 'acc': 0.9084986747913106, 'eval_loss': 0.2672061572255782}


14782.553736686707


In [ ]:
# test_set = train_df[330000:]
# test_set.reset_index(drop=True, inplace=True)
# actual = test_set["labels"].tolist()
# test_set.drop(columns="labels", inplace=True)

predictions, raw_outputs = model.predict(test_set.values.tolist())
# print(predictions)
# print(actual)

# compare = [1 if predictions[i]==actual[i] else 0 for i in range(len(actual))]
# # print(compare)
# print(sum(compare)/float(len(compare)))
# # print(raw_outputs)